In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,66,2024-03-14,1,116.984615,97.960000,0.564985,12.670769,24.913846,0.236108,118.858462,...,10.376923,24.738462,0.191908,123.750769,0,3,3,3,24,2024
1,66,2024-03-14,-1,115.338462,97.209231,0.530185,9.978462,24.821538,0.232108,118.212308,...,10.836364,21.439394,0.221894,119.772727,0,3,3,2,23,2024
2,66,2024-03-14,-1,114.153846,102.378462,0.538385,11.769231,19.761538,0.173215,111.307692,...,10.944615,24.692308,0.203215,113.927692,0,3,3,2,30,2024
3,65,2024-03-14,-2,116.890625,96.789062,0.565438,11.232813,24.495313,0.215234,120.546875,...,10.560000,24.378462,0.187415,115.063077,0,3,3,2,13,2024
4,66,2024-03-14,3,119.138462,99.743077,0.565400,10.524615,22.020000,0.198523,119.433846,...,10.438462,20.730769,0.202000,121.116923,0,3,3,3,7,2024
5,66,2024-03-14,1,112.261538,94.924615,0.536785,11.209231,29.575385,0.196492,118.292308,...,12.543750,26.906250,0.192703,110.454687,0,3,3,2,20,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-14,PHO,BOS,2.850000,1.444444,0
1,2024-03-14,PHI,MIL,3.600000,1.307692,0
2,2024-03-14,WAS,HOU,3.600000,1.307692,0
3,2024-03-14,LAC,CHI,1.420168,2.950000,1
4,2024-03-14,DAL,OKC,5.000000,1.186916,0
5,2024-03-14,NYK,POR,1.186916,5.000000,1


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-14,PHO,BOS,2.850000,1.444444,1
1,2024-03-14,PHI,MIL,3.600000,1.307692,0
2,2024-03-14,WAS,HOU,3.600000,1.307692,0
3,2024-03-14,LAC,CHI,1.420168,2.950000,1
4,2024-03-14,DAL,OKC,5.000000,1.186916,0
5,2024-03-14,NYK,POR,1.186916,5.000000,1


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-14,PHO,BOS,2.850000,1.444444,0
1,2024-03-14,PHI,MIL,3.600000,1.307692,0
2,2024-03-14,WAS,HOU,3.600000,1.307692,0
3,2024-03-14,LAC,CHI,1.420168,2.950000,1
4,2024-03-14,DAL,OKC,5.000000,1.186916,0
5,2024-03-14,NYK,POR,1.186916,5.000000,1


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-14,PHO,BOS,2.850000,1.444444,0
1,2024-03-14,PHI,MIL,3.600000,1.307692,0
2,2024-03-14,WAS,HOU,3.600000,1.307692,0
3,2024-03-14,LAC,CHI,1.420168,2.950000,0
4,2024-03-14,DAL,OKC,5.000000,1.186916,0
5,2024-03-14,NYK,POR,1.186916,5.000000,1


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-14,PHO,BOS,2.850000,1.444444,0.0
1,2024-03-14,PHI,MIL,3.600000,1.307692,0.0
2,2024-03-14,WAS,HOU,3.600000,1.307692,1.0
3,2024-03-14,LAC,CHI,1.420168,2.950000,1.0
4,2024-03-14,DAL,OKC,5.000000,1.186916,1.0
5,2024-03-14,NYK,POR,1.186916,5.000000,1.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-14,PHO,BOS,2.850000,1.444444,0.0
1,2024-03-14,PHI,MIL,3.600000,1.307692,0.0
2,2024-03-14,WAS,HOU,3.600000,1.307692,1.0
3,2024-03-14,LAC,CHI,1.420168,2.950000,1.0
4,2024-03-14,DAL,OKC,5.000000,1.186916,1.0
5,2024-03-14,NYK,POR,1.186916,5.000000,1.0
